In [1]:
cd /content/drive/MyDrive/Colab Notebooks/sign

/content/drive/MyDrive/Colab Notebooks/sign


In [2]:
!unzip sign.zip

Archive:  sign.zip
   creating: dataset/
   creating: dataset/test/
   creating: dataset/test/0/
  inflating: dataset/test/0/IMG_4159.JPG  
  inflating: dataset/test/0/IMG_4725.JPG  
  inflating: dataset/test/0/IMG_4976.JPG  
  inflating: dataset/test/0/IMG_5572.JPG  
  inflating: dataset/test/0/IMG_5695.JPG  
   creating: dataset/test/1/
  inflating: dataset/test/1/IMG_4497.JPG  
  inflating: dataset/test/1/IMG_5309.JPG  
  inflating: dataset/test/1/IMG_5340.JPG  
  inflating: dataset/test/1/IMG_5401.JPG  
  inflating: dataset/test/1/IMG_5992.JPG  
   creating: dataset/test/2/
  inflating: dataset/test/2/IMG_4727.JPG  
  inflating: dataset/test/2/IMG_5342.JPG  
  inflating: dataset/test/2/IMG_5594.JPG  
  inflating: dataset/test/2/IMG_5707.JPG  
  inflating: dataset/test/2/IMG_5718.JPG  
   creating: dataset/test/3/
  inflating: dataset/test/3/IMG_4184.JPG  
  inflating: dataset/test/3/IMG_5114.JPG  
  inflating: dataset/test/3/IMG_5279.JPG  
  inflating: dataset/test/3/IMG_5469.JPG  

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Flatten
from keras.models import Model

In [3]:
train = ImageDataGenerator().flow_from_directory('dataset/train',
                                                 target_size=(224,224),
                                                 batch_size=10)
valid = ImageDataGenerator().flow_from_directory('dataset/valid',
                                                 target_size=(224,224),
                                                 batch_size=30)

Found 1712 images belonging to 10 classes.
Found 300 images belonging to 10 classes.


In [4]:
base_model = VGG16(weights='imagenet',
                   include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg') # pooling은 global의 의미

In [5]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [6]:
for layer in base_model.layers[:-5] :
  layer.trainable = False

In [7]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [8]:
last_output = base_model.output

x = Dense(10, activation='softmax')(last_output)


In [9]:
new_model = Model(inputs=base_model.input, outputs =x)

In [10]:
new_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [11]:
from keras.optimizers import Adam

In [12]:
new_model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy',
                  metrics=['accuracy'])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [13]:
from keras.callbacks import ModelCheckpoint

In [14]:
checkpointer = ModelCheckpoint(filepath='signlanguage.model.hdf5',
                               save_best_only=True)

In [15]:
hist = new_model.fit_generator(train, epochs=20, validation_data=valid, verbose=1, callbacks=[checkpointer])

<ipython-input-15-1291c1032fb9>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = new_model.fit_generator(train, epochs=20, validation_data=valid, verbose=1, callbacks=[checkpointer])


Epoch 1/20
172/172 [==============================] - 25s 121ms/step - loss: 0.7698 - accuracy: 0.7739 - val_loss: 0.1747 - val_accuracy: 0.9500
Epoch 2/20
172/172 [==============================] - 16s 94ms/step - loss: 0.0717 - accuracy: 0.9831 - val_loss: 0.0818 - val_accuracy: 0.9900
Epoch 3/20
172/172 [==============================] - 15s 85ms/step - loss: 0.0069 - accuracy: 0.9982 - val_loss: 0.0858 - val_accuracy: 0.9933
Epoch 4/20
172/172 [==============================] - 15s 85ms/step - loss: 0.0131 - accuracy: 0.9971 - val_loss: 0.0994 - val_accuracy: 0.9933
Epoch 5/20
172/172 [==============================] - 14s 84ms/step - loss: 0.0133 - accuracy: 0.9965 - val_loss: 0.0993 - val_accuracy: 0.9833
Epoch 6/20
172/172 [==============================] - 14s 84ms/step - loss: 0.0211 - accuracy: 0.9947 - val_loss: 0.2112 - val_accuracy: 0.9633
Epoch 7/20
172/172 [==============================] - 15s 86ms/step - loss: 0.0594 - accuracy: 0.9825 - val_loss: 0.1073 - val_accuracy

In [22]:
from sklearn.datasets import load_files
import numpy as np
from keras.utils.np_utils import to_categorical #one-hot encoding

In [17]:
data = load_files('dataset/test')

In [19]:
data['filenames']

array(['dataset/test/5/IMG_5710.JPG', 'dataset/test/2/IMG_5342.JPG',
       'dataset/test/2/IMG_4727.JPG', 'dataset/test/8/IMG_4844.JPG',
       'dataset/test/0/IMG_4976.JPG', 'dataset/test/5/IMG_5250.JPG',
       'dataset/test/7/IMG_5082.JPG', 'dataset/test/6/IMG_4590.JPG',
       'dataset/test/4/IMG_4840.JPG', 'dataset/test/0/IMG_5695.JPG',
       'dataset/test/6/IMG_4942.JPG', 'dataset/test/7/IMG_1176.JPG',
       'dataset/test/5/IMG_4501.JPG', 'dataset/test/6/IMG_5742.JPG',
       'dataset/test/3/IMG_5469.JPG', 'dataset/test/1/IMG_5340.JPG',
       'dataset/test/2/IMG_5718.JPG', 'dataset/test/9/IMG_4495.JPG',
       'dataset/test/9/IMG_5755.JPG', 'dataset/test/5/IMG_5892.JPG',
       'dataset/test/3/IMG_4184.JPG', 'dataset/test/6/IMG_4272.JPG',
       'dataset/test/6/IMG_4731.JPG', 'dataset/test/3/IMG_5114.JPG',
       'dataset/test/8/IMG_4904.JPG', 'dataset/test/4/IMG_4093.JPG',
       'dataset/test/8/IMG_5348.JPG', 'dataset/test/1/IMG_5401.JPG',
       'dataset/test/2/IMG_5707.JP

In [20]:
data['filenames'] #test
data['target'] #target

array([5, 2, 2, 8, 0, 5, 7, 6, 4, 0, 6, 7, 5, 6, 3, 1, 2, 9, 9, 5, 3, 6,
       6, 3, 8, 4, 8, 1, 2, 5, 1, 3, 8, 9, 0, 2, 7, 4, 1, 4, 7, 4, 3, 1,
       7, 9, 0, 0, 9, 8])

In [24]:
print(type(data['filenames']))
print(type(data['target']))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [26]:
paths = np.array(data['filenames'])
targets = to_categorical(data['target'])

In [28]:
test_files, test_target = data['filenames'], targets

In [29]:
from keras.utils import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input

In [30]:
def path_to_tensor(img_path):
  img= load_img(img_path, target_size=(224,224))
  x = img_to_array(img)
  # return x[np.newaxis, :]
  return np.expand_dims(x, axis=0)

In [34]:
def paths_to_tensor(img_paths):
  list_of_tensors = [path_to_tensor(img_path) for img_path in img_paths]
  return np.vstack(list_of_tensors)


In [35]:
test_tensor = paths_to_tensor(test_files)

In [38]:
test_tensor=preprocess_input(test_tensor) #vgg16에 맞게 데이터를 preprocessing

In [39]:
new_model.evaluate(test_tensor, test_target)

2/2 [==============================] - 5s 2s/step - loss: 6.3569e-05 - accuracy: 1.0000


[6.356861558742821e-05, 1.0]

---

In [88]:
img = load_img('/content/drive/MyDrive/Colab Notebooks/sign/examples/test003.jpg', target_size=(224,224))

In [89]:
img = img_to_array(img)

In [90]:
img = img.reshape((1, img.shape[0],img.shape[1],img.shape[2]))

In [91]:
img = preprocess_input(img)

In [92]:
pred = new_model.predict(img)

1/1 [==============================] - 0s 17ms/step


In [93]:
np.argmax(pred)

9

In [94]:
32784 * 16* 16

8392704

In [95]:
32784 * 2048

67141632

In [96]:
2e-5

2e-05

In [97]:
0.00002

2e-05